In [2]:
#This file reads the note_csv, spectrums. Generate and save the numpy arrays which are inputs to the model

In [3]:
import numpy as np
import glob
import random
import copy

In [4]:
spectrums = 'E:/notes_database/spectrograms/'
files = glob.glob('E:/notes_database/index/*')

In [5]:
def parse_row(csv_row):  # same as generate_database.ipynb

    csv_row_array = csv_row.split(",")

    filename = csv_row_array[0]
    time_array = [float(t) for t in csv_row_array[1::2]]
    note_array = [int(x) for x in csv_row_array[2::2]]

    assert len(time_array) == len(note_array)

#     print(time_array)
#     print(note_array)

    return filename, time_array, note_array

In [6]:
def parse_csv(csv_file): # Convert the csv files into numpy arrays
    
    with open(csv_file, 'r') as csvfile:
        csv_rows = csvfile.read().split("\n")

    filenames = []
    time_arrays = []
    note_arrays = []
    
    for csv_row in csv_rows:
        filename, time_array, note_array = parse_row(csv_row)
        
        filenames.append(filename)
        time_arrays.append(time_array)
        note_arrays.append(note_array)
    
    return filenames,time_arrays,note_arrays

In [7]:
def lstminput(previous_time, y_train, random_units, one_row):
    lstm_input = np.zeros((20, 24))
    length_of_time =  (20 * 27 + random_units)* 0.006
    
    for num in range(len(previous_time)):
        if previous_time[-1] - previous_time[num] - random_units< length_of_time:
            checkpoint = num
            break
                        
    start_time = previous_time[-1] - length_of_time
    if start_time < 0:
        start_time = 0
                
    for num in range(checkpoint, len(y_train)):
        position = int(((previous_time[num]-start_time-random_units)//0.006)//27) 
        lstm_input[position] = y_train[num]
    
    counter_lstm_input = copy.copy(lstm_input)
    counter_lstm_input = np.append(counter_lstm_input, one_row.reshape((1, 24)), axis=0)
    counter_lstm_input = counter_lstm_input[1:, :]
    return lstm_input, counter_lstm_input

In [8]:
def prepare_inputs(csv_name):
    # the time for a unit width of spectrum is about 0.003 second
    # however a unit in time_array is 0.5 second, so to calculate the unit numbers, should divide the time_number by 0.006
    filenames,time,note = parse_csv(csv_name)
    lstm_all_inputs = []
    counter_lstm_all_inputs = []
    cnn_all_inputs = []
    counter_cnn_all_inputs = []
    y_all_train = []
    counter_y_all_train = []
    for file in range(len(filenames)):
        one_row = np.zeros(24)
        previous_time = [time[file][0]]
        spectrum = np.load(spectrums+filenames[file]+'.npy')
        y_train = []
        # load the spectrum numpy array from (81+random_units) before the timing(onset) to (32-random) units after the timing
        # then load a counter example to show the notes don't belong to the next frame
        for i in range(len(time[file])):
            if time[file][i] == previous_time[-1]:
                one_row[note[file][i]] = 1
                
            else:
                random_units = random.randint(0, 27)
                
                #append lstm inputs
                lstm_input, counter_lstm_input = lstminput(previous_time, y_train, random_units, one_row)
                lstm_all_inputs.append(lstm_input)
                counter_lstm_all_inputs.append(counter_lstm_input)
                
                #append y_train
                y_train.append(one_row)
                y_all_train.append(one_row)
                counter_y_all_train.append(np.zeros(24))
                
                #append cnn inputs
                #the timing+20 is about the timing of energy peak
                cnn_input = spectrum[:, int(previous_time[-1]//0.006-random_units-81+20):int(previous_time[-1]//0.006+27-random_units+20)]
                counter_cnn_input = spectrum[:, int(previous_time[-1]//0.006+27-random_units+20):int(previous_time[-1]//0.006+135-random_units+20)]
                    
                cnn_all_inputs.append(cnn_input)
                counter_cnn_all_inputs.append(counter_cnn_input)
                
                #check if the last cnn_input has right shape
                if cnn_all_inputs[-1].shape != (108, 108):
                    cnn_all_inputs = cnn_all_inputs[:-1]
                    lstm_all_inputs = lstm_all_inputs[:-1]
                    y_all_train = y_all_train[:-1]
                
                if counter_cnn_all_inputs[-1].shape != (108, 108):
                    counter_cnn_all_inputs = counter_cnn_all_inputs[:-1]
                    counter_lstm_all_inputs = counter_lstm_all_inputs[:-1]
                    counter_y_all_train = counter_y_all_train[:-1]
                
                one_row = np.zeros(24)
                one_row[note[file][i]] = 1
                previous_time.append(time[file][i])
            
            if i == len(time[file])-1:
                lstm_input, counter_lstm_input = lstminput(previous_time, y_train, random_units, one_row)
                lstm_all_inputs.append(lstm_input)
                
                #append y_train
                y_train.append(one_row)
                y_all_train.append(one_row)
                
                #append cnn input
                cnn_input = spectrum[:, int(previous_time[-1]//0.006-random_units-81+20):int(previous_time[-1]//0.006+27-random_units+20)]
                cnn_all_inputs.append(cnn_input)
                
        #check if the last cnn_input has right shape
        if cnn_all_inputs[-1].shape != (108, 108):
            cnn_all_inputs = cnn_all_inputs[:-1]
            lstm_all_inputs = lstm_all_inputs[:-1]
            y_all_train = y_all_train[:-1]
        
        if counter_cnn_all_inputs[-1].shape != (108, 108):
            counter_cnn_all_inputs = counter_cnn_all_inputs[:-1]
            counter_lstm_all_inputs = counter_lstm_all_inputs[:-1]
            counter_y_all_train = counter_y_all_train[:-1]
        
    lstm_all_inputs = np.array(lstm_all_inputs)
    counter_lstm_all_inputs = np.array(counter_lstm_all_inputs)
    cnn_all_inputs = np.array(cnn_all_inputs).reshape(len(cnn_all_inputs), 108, 108, 1)
    counter_cnn_all_inputs = np.array(counter_cnn_all_inputs).reshape(len(counter_cnn_all_inputs), 108, 108, 1)
    y_all_train = np.array(y_all_train)
    counter_y_all_train = np.array(counter_y_all_train)
    
    return lstm_all_inputs, cnn_all_inputs, y_all_train, counter_lstm_all_inputs, counter_cnn_all_inputs, counter_y_all_train

In [9]:
save = 'E:/notes_database/75_overlap/'

In [10]:
count = 0
number = 0
lstm_save, cnn_save, y_train_save, clstm_save, ccnn_save, cytrain_save = prepare_inputs(files[0])
for i in range(len(files))[1:]:
    if number < 5:
        lstm, cnn, y_train, c_lstm, c_cnn, c_ytrain = prepare_inputs(files[i])
        lstm_save = np.append(lstm_save, lstm, axis=0)
        cnn_save = np.append(cnn_save, cnn, axis=0)
        y_train_save = np.append(y_train_save, y_train, axis=0)
        clstm_save = np.append(clstm_save, c_lstm, axis=0)
        ccnn_save = np.append(ccnn_save, c_cnn, axis=0)
        cytrain_save = np.append(cytrain_save, c_ytrain, axis=0)
        
        number += 1
        
    elif number >= 5:
        if y_train_save.shape[0] != lstm_save.shape[0]:
            print('y_train_error_{}_{}'.format(y_train_save.shape[0], lstm_save.shape[0]))
        if cnn_save.shape[0] != lstm_save.shape[0]:
            print('cnn_error_{}_{}'.format(cnn_save.shape[0], lstm_save.shape[0]))
        
        np.save(save+'lstm_inputs/{}.npy'.format(count), lstm_save.astype(np.int16))
        np.save(save+'cnn_inputs/{}.npy'.format(count), cnn_save.astype(np.float16))
        np.save(save+'y_train/{}.npy'.format(count), y_train_save.astype(np.int16))
        np.save(save+'counter_lstm_inputs/{}.npy'.format(count), clstm_save.astype(np.int16))
        np.save(save+'counter_cnn_inputs/{}.npy'.format(count), ccnn_save.astype(np.float16))
        np.save(save+'counter_y_train/{}.npy'.format(count), cytrain_save.astype(np.int16))
        lstm_save, cnn_save, y_train_save, clstm_save, ccnn_save, cytrain_save = prepare_inputs(files[i])
        
        count += 1
        number = 1
        
np.save(save+'lstm_inputs/{}.npy'.format(count), lstm_save.astype(np.int16))
np.save(save+'cnn_inputs/{}.npy'.format(count), cnn_save.astype(np.float16))
np.save(save+'y_train/{}.npy'.format(count), y_train_save.astype(np.int16))
np.save(save+'counter_lstm_inputs/{}.npy'.format(count), clstm_save.astype(np.int16))
np.save(save+'counter_cnn_inputs/{}.npy'.format(count), ccnn_save.astype(np.float16))
np.save(save+'counter_y_train/{}.npy'.format(count), cytrain_save.astype(np.int16))